<a href="https://colab.research.google.com/github/hammad93/hurricane-tts/blob/main/hurricane_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation
Please make sure to run all tests.

In [1]:
%cd ~/
!git clone https://github.com/hammad93/hurricane-tts.git

/root
fatal: destination path 'hurricane-tts' already exists and is not an empty directory.


In [2]:
!pip install -r hurricane-tts/requirements.txt

In [1]:
import os
if not os.getenv("AZURE_OPENAI_API_KEY") :
  from google.colab import userdata
  os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
if not os.getenv("AZURE_REDIS_KEY") :
  os.environ["AZURE_REDIS_KEY"] = userdata.get('AZURE_REDIS_KEY')

In [2]:
%cd ~/hurricane-tts
!python ~/hurricane-tts/test.py

/root/hurricane-tts
.Here are the constructed messages: [{'role': 'system', 'content': 'You are an AI assistant that helps people find information.'}, {'role': 'user', 'content': 'test'}]
..
----------------------------------------------------------------------
Ran 3 tests in 8.704s

OK


In [3]:
import prompts
import utils
import db
# we generate prompts by ingesting live hurricane data and supported languages
storm_data = utils.transform_storm_data()
prompt_data = prompts.generate_prompts()

# metadata related to multilingual generation
supported_langs = prompts.unique_lang_list()
supported_langs_df = utils.tts_langs()

In [4]:
# example
print(prompt_data['storms'][0])

Provided are the tropical storm records with geographic coordinates and wind speed in knots.
Please respond with at least 2 of the supported languages by utilizing language geography and the storm's latitude and longitude.
The response must be a Python list meant to be input as a string into eval(). Please make sure the languages are in the supported list.
As an example, ["Chinese, Mandarin", "Haitian Creole", "Indonesian"]
[{'type': 'history', 'lat': 13.5, 'lon': 87.0, 'time': '2023-11-14 18:00:00', 'wind_speed': 20}, {'type': 'history', 'lat': 14.1, 'lon': 86.9, 'time': '2023-11-15 00:00:00', 'wind_speed': 20}, {'type': 'history', 'lat': 14.8, 'lon': 86.8, 'time': '2023-11-15 06:00:00', 'wind_speed': 20}, {'type': 'history', 'lat': 15.4, 'lon': 86.7, 'time': '2023-11-15 12:00:00', 'wind_speed': 20}, {'type': 'history', 'lat': 16.3, 'lon': 86.8, 'time': '2023-11-15 18:00:00', 'wind_speed': 25}, {'type': 'history', 'lat': 17.3, 'lon': 87.2, 'time': '2023-11-16 00:00:00', 'wind_speed': 

# Language Geography inference from tropical storm geographical coorindates
We're trying to answer the question, "Which languages do we produce speech for to report on this tropical storm?" This pipeline will produce an English output but also utilize massively multilingual capabilities such that emergency notices are also in local languages

In [7]:
storm_langs = {} # keys are storm id's and values are the languages
storm_chats = {} # stores chat histories for storms
for index, storm_prompt in enumerate(prompt_data['storms']) :
  storm_id = list(storm_data.keys())[index]
  # Get the languages to generate the report for this storm
  retries = 10 # sometimes it fails, so we retry it
  while retries > 0 :
    response = prompts.chat(system=prompt_data['system'], message=storm_prompt)
    result = utils.llm_response_transform(
        resp=response,
        supported_langs=supported_langs)
    if result :
      print(f'Successfully parsed {response}')
      break
    else :
      retries = retries - 1
      print(f"Failed. Retries left: {retries}")
  if retries < 1 :
    raise Exception("Couldn't produce a correct output from LLM.")

  # store results
  storm_langs[storm_id] = {'names': ['English'] + result}
  storm_chats[storm_id] = {
      'history' : [{'role': 'system', 'content': prompt_data['system']},
                   {'role': 'user', 'content': storm_prompt},
                   {'role': 'assistant', 'content': response}]
  }

Here are the constructed messages: [{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nThis is the list of supported languages,\n{'Chinese, Min Nan', 'Ngâ\\x80\\x99akarimojong', 'Wayana', 'Chinantec, Comaltepec', 'Welsh', 'Alangan', 'Kankanaey', 'Siang', 'Mankanya', 'Lobala', 'Marathi', 'Cora, El Nayar', 'Dida, YocobouÃ©', 'Krumen, Tepo', 'Birifor, Malba', 'Kiribati', 'Rajbanshi', 'Mixtec, Jamiltepec', 'Chin, Hakha', 'Urdu', 'Acholi', 'Fon', 'Deg', 'Ndogo', 'Mbuko', 'Lokaa', 'Amis', 'Jakalteko', 'Ganda', 'Konni', 'MÃ©nik', 'Daasanach', 'Cebuano', 'Ifugao, Amganad', 'Kigiryama', 'Matal', 'Mari, Meadow', 'Ossetic', 'Chuvash', 'Urarina', 'Zapotec, YalÃ¡lag', 'Malvi', 'Markweeta', 'Nzema', 'Gen', 'Guayabero', 'Quechua, Huaylas Ancash', 'Quechua, North JunÃ\\xadn', 'Tolaki', 'BorÃ´ro', 'Klon', 'Siwu', 'Lao', 'TriÃ³', 'Pidgin, Nigerian', 'Gapapaiwa', 'Ngaju', 'Sadri', 'Zapotec, Choapan', 'Icelandic', 'Mixtec, Magdalena PeÃ±asco', 'Jur Mod

# Tropical Storm Report
This code will create the report in the language specified based on the tropical storm.

In [8]:
with open('prompts/report-prompt.txt', 'r') as file:
  report_prompt = file.read()
for storm in storm_langs:
  print(storm)
  reports = []
  for lang in storm_langs[storm]['names']:
    print(storm_chats[storm]['history'])
    # Construct the prompt
    message = report_prompt.format(lang=lang, id=storm[2:4])
    print(message)
    # Send to LLM
    response = prompts.chat(
        message = message, history = storm_chats[storm]['history'])
    print(response)
    # store data
    reports.append(response)
  storm_langs[storm]['reports'] = reports

IO72023
[{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nThis is the list of supported languages,\n{'Chinese, Min Nan', 'Ngâ\\x80\\x99akarimojong', 'Wayana', 'Chinantec, Comaltepec', 'Welsh', 'Alangan', 'Kankanaey', 'Siang', 'Mankanya', 'Lobala', 'Marathi', 'Cora, El Nayar', 'Dida, YocobouÃ©', 'Krumen, Tepo', 'Birifor, Malba', 'Kiribati', 'Rajbanshi', 'Mixtec, Jamiltepec', 'Chin, Hakha', 'Urdu', 'Acholi', 'Fon', 'Deg', 'Ndogo', 'Mbuko', 'Lokaa', 'Amis', 'Jakalteko', 'Ganda', 'Konni', 'MÃ©nik', 'Daasanach', 'Cebuano', 'Ifugao, Amganad', 'Kigiryama', 'Matal', 'Mari, Meadow', 'Ossetic', 'Chuvash', 'Urarina', 'Zapotec, YalÃ¡lag', 'Malvi', 'Markweeta', 'Nzema', 'Gen', 'Guayabero', 'Quechua, Huaylas Ancash', 'Quechua, North JunÃ\\xadn', 'Tolaki', 'BorÃ´ro', 'Klon', 'Siwu', 'Lao', 'TriÃ³', 'Pidgin, Nigerian', 'Gapapaiwa', 'Ngaju', 'Sadri', 'Zapotec, Choapan', 'Icelandic', 'Mixtec, Magdalena PeÃ±asco', 'Jur Modo', 'Gujarati', 'Carib', 'P

In [9]:
storm_langs

{'IO72023': {'names': ['English', 'Bengali', 'Hindi'],
  'reports': ['"A tropical storm, known as storm number 72, is expected to hit our area. Please take immediate precautions and prepare for strong winds, heavy rainfall, and potential flooding. Let\'s take a moment to remember similar storms that have affected our community in the past and the steps we took to recover. Stay safe and keep updated on the latest weather information from local authorities."',
   '"আপনার এলাকার মহাসাগর প্রদূষণ সেবে একটি বিশেষ সংবাদ। স্টর্ম নম্বর 72 একটি তীব্র সংকটের প্রস্তাব। আমরা একটি সম্পূর্ণ তথ্যপূর্ণ রিপোর্ট সরবরাহ করতে সমর্থ। এটি আপনার সুরক্ষা সম্পর্কে জেনে নিন এবং আবশ্যক ক্ষেত্রে উচ্চ জ্বরমুক্ত সাধন সমূহ স্থাপন করুন। একটি সমস্যা হতে পারে ভবিষ্যতে আমরা এমন অতিরিক্ত একটি স্টর্ম আশা করছিলাম যা পিছনে আসছে এবং একটি সাবস্ক্রাইব বৈশিষ্ট্য পেতে চাইলে বা আ',
   'आगे बढ़कर प्रधानमंत्री ने स्वच्छ भारत अभियान की केंद्रित शक्ति और सभी उनके सहयोगियों के साथ काम करने की अपील की। स्टार्म नंबर 72 देश में आने वाले न

The following are from the TTS implementation
https://github.com/facebookresearch/fairseq/blob/main/examples/mms/tts/tutorial/MMS_TTS_Inference_Colab.ipynb


In [10]:
%cd ~/
!git clone https://github.com/jaywalnut310/vits.git
%cd vits/

!pip install Cython==0.29.21
!pip install librosa==0.8.0
!pip install phonemizer==2.2.1
!pip install scipy
!pip install "numpy<1.24"
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install Unidecode==1.1.1

%cd monotonic_align/
%mkdir monotonic_align
!python3 setup.py build_ext --inplace
%cd ../
%pwd

/root
Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Receiving objects: 100% (81/81), 3.33 MiB | 6.41 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/root/vits
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.5
    Uninstalling Cython-3.0.5:
      Successfully uninstalled Cython-3.0.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.6 MB/s eta 0:00:00
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201367 sha256=86b5e6c38c387e75195fe40686ce41f3facfa1eefb1241b10778ba9fe286695f
  Stored in directory: /root/.cache/pip/wheels/bf/b7/85/2f8044306ccec014930aea23ad4852fca9e2584e21c6972bc6
Successfully built librosa
  Attempting uninstall: librosa
   

'/root/vits'

In [11]:
# we have the data we need and the utils class is also used in the next code
import sys
del sys.modules["utils"]

In [12]:
%cd ~/hurricane-tts/
import fair_mms_tts
%cd ~/vits
import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn

/root/hurricane-tts
/root/vits


In [13]:
# match language names to iso code
for storm in storm_langs :
  langs = storm_langs[storm]['names']
  print(langs)
  codes = [supported_langs_df[
      supported_langs_df['Language Name'] == name
  ]['ISO 693-3'].iloc[0] for name in langs]
  print(codes)
  storm_langs[storm]['codes'] = codes

['English', 'Bengali', 'Hindi']
['eng', 'ben', 'hin']
['English', 'Tagalog', 'Indonesian']
['eng', 'tgl', 'ind']
['English', 'Indonesian', 'Samoan']
['eng', 'ind', 'smo']
['English', 'Spanish', 'Portuguese']
['eng', 'spa', 'por']
['English', 'Tagalog', 'Chinantec, Lealao']
['eng', 'tgl', 'cle']
['English', 'Indonesian', 'Chinantec, Comaltepec']
['eng', 'ind', 'cco']
['English', 'Spanish', 'Bengali']
['eng', 'spa', 'ben']
['English', 'Spanish', 'Kalinga, Lubuagan']
['eng', 'spa', 'knb']


In [14]:
import soundfile as sf
def generate_audio(report, lang_code, out):
  '''
  Generates the audio given the report and lang_code
  '''
  ckpt_dir = fair_mms_tts.download(lang_code)
  import torch
  if torch.cuda.is_available():
      device = torch.device("cuda")
  else:
      device = torch.device("cpu")
  print(device)
  print(f"Run inference with {device}")
  vocab_file = f"{ckpt_dir}/vocab.txt"
  config_file = f"{ckpt_dir}/config.json"
  assert os.path.isfile(config_file), f"{config_file} doesn't exist"
  hps = utils.get_hparams_from_file(config_file)

  text_mapper = fair_mms_tts.TextMapper(vocab_file)
  net_g = SynthesizerTrn(
      len(text_mapper.symbols),
      hps.data.filter_length // 2 + 1,
      hps.train.segment_size // hps.data.hop_length,
      **hps.model)
  net_g.to(device)
  _ = net_g.eval()

  g_pth = f"{ckpt_dir}/G_100000.pth"
  print(f"load {g_pth}")

  _ = utils.load_checkpoint(g_pth, net_g, None)
  txt = fair_mms_tts.preprocess_text(report, text_mapper, hps, lang=lang_code)
  stn_tst = text_mapper.get_text(txt, hps)

  with torch.no_grad():
      x_tst = stn_tst.unsqueeze(0).to(device)
      x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
      hyp = net_g.infer(
          x_tst, x_tst_lengths, noise_scale=.667,
          noise_scale_w=0.8, length_scale=1.0
      )[0][0,0].cpu().float().numpy()

  print(f"Generated audio")
  sf.write(out, hyp, hps.data.sampling_rate)

In [15]:
# generate audio
import time
time_fname = int(time.time())
outputs = []
for storm in storm_langs :
  current = storm_langs[storm]
  for index, code in enumerate(current['codes']):
    path = f"/root/{storm}_{time_fname}_{current['names'][index]}.wav"
    outputs.append(path)
    generate_audio(report = current['reports'][index],
                   lang_code = code,
                   out = path)

Download model for language: eng
Model checkpoints in ./eng: ['G_100000.pth', 'vocab.txt', 'config.json']
cuda
Run inference with cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


load ./eng/G_100000.pth
eng
text after filtering OOV: a tropical storm known as storm number 2 is expected to hit our area please take immediate precautions and prepare for strong winds heavy rainfall and potential flooding let's take a moment to remember similar storms that have affected our community in the past and the steps we took to recover stay safe and keep updated on the latest weather information from local authorities
Generated audio
Download model for language: ben
Model checkpoints in ./ben: ['G_100000.pth', 'vocab.txt', 'config.json']
cuda
Run inference with cuda
load ./ben/G_100000.pth
ben
text after filtering OOV: আপনার এলাকার মহাসাগর প্রদূষণ সেবে একটি বিশেষ সংবাদ স্টর্ম নম্বর 72 একটি তীব্র সংকটের প্রস্তাব আমরা একটি সম্পূর্ণ তথ্যপূর্ণ রিপোর্ট সরবরাহ করতে সমর্থ এটি আপনার সুরক্ষা সম্পর্কে জেনে নিন এবং আবশ্যক ক্ষেত্রে উচ্চ জ্বরমুক্ত সাধন সমূহ স্থাপন করুন একটি সমস্যা হতে পারে ভবিষ্যতে আমরা এমন অতিরিক্ত একটি স্টর্ম আশা করছিলাম যা পিছনে আসছে এবং একটি সাবস্ক্রাইব বৈশিষ্ট্য পেত

In [16]:
import db
import json
print(outputs)
db.upload_latest_audios(outputs)

['/root/IO72023_1700191263_English.wav', '/root/IO72023_1700191263_Bengali.wav', '/root/IO72023_1700191263_Hindi.wav', '/root/WP172023_1700191263_English.wav', '/root/WP172023_1700191263_Tagalog.wav', '/root/WP172023_1700191263_Indonesian.wav', '/root/SH22023_1700191263_English.wav', '/root/SH22023_1700191263_Indonesian.wav', '/root/SH22023_1700191263_Samoan.wav', '/root/AL222023_1700191263_English.wav', '/root/AL222023_1700191263_Spanish.wav', '/root/AL222023_1700191263_Portuguese.wav', '/root/WP972023_1700191263_English.wav', '/root/WP972023_1700191263_Tagalog.wav', '/root/WP972023_1700191263_Chinantec, Lealao.wav', '/root/WP982023_1700191263_English.wav', '/root/WP982023_1700191263_Indonesian.wav', '/root/WP982023_1700191263_Chinantec, Comaltepec.wav', '/root/IO942023_1700191263_English.wav', '/root/IO942023_1700191263_Spanish.wav', '/root/IO942023_1700191263_Bengali.wav', '/root/AL982023_1700191263_English.wav', '/root/AL982023_1700191263_Spanish.wav', '/root/AL982023_1700191263_Ka

In [17]:
!git -C /root/hurricane-tts/ pull

Already up to date.
